In [1]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head()


melb_df['Date']= pd.to_datetime(melb_df['Date'])
melb_df['quarter'] = melb_df['Date'].dt.quarter

melb_df['quarter'].value_counts()
melb_df= melb_df.drop(['quarter'], axis=1)


columns_excl=['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
uniqs=150
for col in melb_df.columns:
    if melb_df[col].nunique() < uniqs and col not in columns_excl:
        melb_df[col]= melb_df[col].astype('category')



'''    Произведите сортировку столбца AreaRatio по убыванию. При этом индексы полученной таблицы замените на новые. Какое значение площади здания находится в строке 1558?     '''

#melb_df.sort_values(by='AreaRatio', ascending=False, ignore_index=True).loc[1558]  

'''   Найдите таунхаусы (Type) с количеством жилых комнат (Rooms) больше 2. Отсортируйте полученную таблицу сначала по возрастанию числа комнат,
а затем по убыванию средней площади комнат (MeanRoomsSquare). Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18?    '''

mask1 = melb_df['Rooms'] >2
mask2 = melb_df['Type']=='townhouse'
melb_df[mask1 & mask2].sort_values(by=['Rooms','MeanRoomsSquare'], ascending=[True, False], ignore_index=True).loc[18]
 
 
melb_df.groupby('MonthSale')['Price'].agg(['count', 'mean', 'max']).sort_values(by='count', ascending=False) # построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж, а также среднее и максимальное значения цен объектов недвижимости (Price), сгруппированных по номеру месяца продажи (MonthSale). Результат отсортируем по количеству продаж в порядке убывания


melb_df.groupby('Regionname')['Lattitude'].std().sort_values(ascending=True)
#melb_df['Date']= pd.to_datetime(melb_df['Date'])

date_mask1=pd.to_datetime('2017-5-1')
date_mask2=pd.to_datetime('2017-9-1')
mask3=(date_mask1<=melb_df['Date']) & (melb_df['Date']<=date_mask2)
melb_df[mask3].groupby('SellerG')['Price'].sum().sort_values(ascending=True)

melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc=['median'],
    fill_value=0).sort_values(by=('median', 'unit'), ascending=False)

median                  
Type             house townhouse    unit
SellerG                                 
Nick           2025000    780000  900000
Marshall       1975000   1408500  715000
Cayzer         1505000   1450000  707500
Kay            2220000   1365000  695000
Noel           1400500    990000  693250
Buxton         1323750   1030000  670000
Fletchers      1390000   1238000  653000
Chisholm       1520000    950000  640000
Philip         1035000    701000  636000
RT             1640000   1400000  630000
C21             836500    768750  625500
RW              870000    630500  625500
Moonee          806000    715000  622000
O'Brien         737504    430000  618500
Hodges         1500000   1105000  605500
Eview           760000    597500  602000
Woodards       1261000    865500  600000
Collins        1566500    930000  592000
Jellis         1505000   1052000  591750
Purplebricks    810000    927500  582500
Williams       1161000    735000  577000
hockingstuart  1055000    935000  566000
Alexkarbon     1036250    935000  560000
Miles          1090000    721500  555000
Buckingham      830000    820000  553000
Gary           1370000    993500  550000
Biggin         1151750    675000  548000
Greg           1330000    900000  547500
McGrath        1250000    810000  542500
other           901000    840500  535000
Nelson         1055000    730000  530000
HAR             695000         0  518000
McDonald       1242500         0  502500
Barry           830000    727500  502000
Harcourts       740000    653750  500000
Ray             900000    650000  497500
Thomson        1177000    500000  495000
Douglas         684000    500000  494500
Brad            875000    700000  465000
Rendina        1190000    750000  459000
Bells           692500    430500  457500
Jas             935000    699000  455000
Raine           745000    608000  447500
LITTLE          970000    800000  447000
YPA             596250    498500  443500
Love            780000    846000  440000
Village         973500    727500  420000
Stockdale       715500    550000  410000
Sweeney         850000    728000  381000
Burnham         922500         0  305000

In [122]:
'''   Первичный анализ данных и конкатенация(объединение)   '''
import pandas as pd

movies=pd.read_csv('data/movies.csv')
ratings1=pd.read_csv('data/ratings1.csv')
ratings2=pd.read_csv('data/ratings2.csv')
dates=pd.read_csv('data/dates.csv')

#ratings['userId'].nunique()
dates['date'] = pd.to_datetime(dates['date'])
dates['date'].dt.year.value_counts()

'''  

        objs — список объектов DataFrame ([df1, df2,…]), которые должны быть сконкатенированы;
        axis — ось определяет направление конкатенации: 0 — конкатенация по строкам (по умолчанию), 1 — конкатенация по столбцам;
        join — либо inner (пересечение), либо outer (объединение); рассмотрим этот момент немного позже;
        ignore_index — по умолчанию установлено значение False, которое позволяет значениям индекса оставаться такими, какими они были в исходных данных. Если установлено значение True, параметр будет игнорировать исходные значения и повторно назначать значения индекса в последовательном порядке.

  '''


ratings = pd.concat([ratings1, ratings2], ignore_index=True)
#display(ratings)

#print('Число строк в таблице ratings: ', ratings.shape[0])
#print('Число строк в таблице dates: ', dates.shape[0])
#print(ratings.shape[0] == dates.shape[0])  
''' в абзаце выше при выгрузке данных информация об оценках какого-то пользователя попала в обе таблицы (ratings1 и ratings2). В результате конкатенации случилось дублирование строк.'''

ratings = ratings.drop_duplicates(ignore_index=True) #Дропаем дубликаты. Не забываем обновить индексы после удаления дублей, выставив параметр ignore_index=True
#print('Число строк в таблице ratings: ', ratings.shape[0])

ratings_dates = pd.concat([ratings, dates], axis=1)  # Объединили таблицу с датами и рейтингами
#display(ratings_dates.head())




'''  .join()

        other — таблица, которую мы присоединяем. При объединении она является «правой», а исходная таблица, от имени которой вызывается метод, является «левой».
        how — параметр типа объединения. Он может принимать значения 'inner', 'left' (left outer), 'right' (right outer), и 'outer' (full outer). По умолчанию параметр установлен на 'left'.
        on — параметр, который определяет, по какому столбцу в «левой» таблице происходит объединение по индексам из «правой».
        lsuffix и rsuffix — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.

'''
joined = ratings_dates.join(         
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
#display(joined.head())




'''     .merge()

        right — присоединяемая таблица. По умолчанию она является «правой».
        how — параметр типа объединения. По умолчанию принимает значение 'inner'.
        on — параметр, который определяет, по какому столбцу происходит объединение. Определяется автоматически, но рекомендуется указывать вручную.
        left_on — если названия столбцов в «левой» и «правой» таблицах не совпадают, то данный параметр отвечает за наименования ключевого столбца исходной таблицы.
        right_on — аналогично предыдущему, параметр отвечает за наименование ключевого столбца присоединяемой таблицы.
        lsuffix и rsuffix — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.

'''
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
#display(merged.head())


merge_ratings = ratings1.merge(ratings2, how='outer')   # обрати внимание, при склейке таблиц с помощью merge пропали дубликаты, которые возникают при concat(). Фича!
#print('Число строк в таблице merge_ratings: ', merge_ratings.shape[0])
#display(merge_ratings)

In [108]:
'''   Practice   '''

import pandas as pd

#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None


ratings_movies=pd.read_csv('data/ratings_movies.csv')
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
ratings_movies.drop(['Unnamed: 0'], axis=1)

#ratings_movies.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(by=['count', 'mean'], ascending=[True,False])
ratings_movies['date']=pd.to_datetime(ratings_movies['date'])


mask = ratings_movies['year_release'] == 2018.0
genr = ratings_movies[mask].groupby('genres')['rating'].agg(
                                                            ['count', 'mean']
                                                            )
genr[genr['count']>10].sort_values(
                        by=['mean', 'count'], ascending=False
                                )

ratings_movies['year_rating']=ratings_movies['date'].dt.year
ratings_table=ratings_movies.pivot_table(
        values='rating',
        index='genres',
        columns='year_rating',
        aggfunc='mean',
        fill_value=0
)
#ratings_table.loc['Comedy']


In [ ]:

import pandas as pd

#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None


ratings_movies=pd.read_csv('data/ratings_movies.csv')
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
ratings_movies.drop(['Unnamed: 0'], axis=1)

#ratings_movies.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(by=['count', 'mean'], ascending=[True,False])
ratings_movies['date']=pd.to_datetime(ratings_movies['date'])


mask = ratings_movies['year_release'] == 2018.0
genr = ratings_movies[mask].groupby('genres')['rating'].agg(
                                                            ['count', 'mean']
                                                            )
genr[genr['count']>10].sort_values(
                        by=['mean', 'count'], ascending=False
                                )

ratings_movies['year_rating']=ratings_movies['date'].dt.year
ratings_table=ratings_movies.pivot_table(
        values='rating',
        index='genres',
        columns='year_rating',
        aggfunc='mean',
        fill_value=0
)
#ratings_table.loc['Comedy']


In [144]:
orders=pd.read_csv('data/orders.csv', sep=';')
products=pd.read_csv('data/products.csv', sep=';')

products['ID товара']=products['Product_ID']
products=products.drop(['Product_ID'], axis=1)
ord_prd=orders.merge(products, how='inner')
ord_prd

#ord_prd['total_order_value']=ord_prd['Количество']*ord_prd['Price']
#ord_prd.groupby('ID Покупателя')['total_order_value'].agg('sum').sort_values(ascending=False)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,"Носки Подарочные, муж",199,RUR
1,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,"Носки Подарочные, муж",199,RUR
2,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,"Носки Подарочные, муж",199,RUR
3,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,"Носки Простые, муж",45,RUR
4,08.11.2019 08:36:20,1,5,"Оплачен, формируется к отправке",Да,Нет,Нет,86,1,"Носки Простые, муж",45,RUR
5,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,"Носки Подарочные, жен",249,RUR
6,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,"Носки Подарочные, жен",249,RUR
7,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,"Носки Подарочные, жен",249,RUR
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,"Носки Подарочные, жен",249,RUR
9,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,Носки беговые Camino,999,RUR
